small naver news로부터 명사로 이뤄진 term frequency matrix를 만든 뒤, L1 regularization logistic regression을 이용하여 키워드를 추출합니다. 

명사 추출을 위해서 soy.git 의 LRNounExtractor를 이용합니다.

LRNounExtractor는 extract()의 return으로 (현재는) 
    
    nouns: {word:(명사점수, 알려진 R set의 비율)}
    cohesions: CohesionProbability class
    
두 가지를 return 합니다. 아직 확정된 버전이 아니어서 return type이 바뀔 수 있습니다. 

In [1]:
import sys
sys.path.append('../mypy/')
from corpus import Corpus

corpus_path = '../../../data/small_naver_news/processed/corpus.txt'
corpus = Corpus(corpus_path, iter_sent=True)

In [2]:
import sys
sys.path.append('../soy/')
from soy.nlp.tags import LRNounExtractor

noun_extractor = LRNounExtractor()
nouns, cohesions = noun_extractor.extract(corpus)

Use default r_score_file
2398 r features was loaded
scanning completed################################ (99.921 %)
(L,R) has (6174, 3295) tokens
building lr-graph completed################################ (99.921 %)

noun score threshold 는 0.2 이상, min count는 10이상인 명사만을 선택하여 custom_tokenizer를 만듦니다. 

In [4]:
min_count = 10
noun_score_threshold = 0.2

noun_set = {word for word, score in nouns.items() 
             if (score[0] > noun_score_threshold) 
             and (cohesions.L[word] > min_count) }

print('%d --> %d nouns are selected' % (len(nouns), len(noun_set)))

1683 --> 1616 nouns are selected


## Term frequency matrix 만들기

parse_noun은 주어진 token에 대하여 어절의 왼쪽에 명사가 존재할 경우 이를 잘라내주는 함수입니다. 만약 '뉴스기자가'라는 어절에 대하여 '뉴스'와 '뉴스기자' 두 명사가 모두 존재한다면 길이가 더 긴 '뉴스기자'를 return 합니다. 이를 위하여 reversed(range)를 이용하여 길이의 역순으로 명사를 선택하였습니다. 

문서가 주어지면 추출된 명사만을 출력하는 custom_tokenize를 만들었습니다. 이를 이용하면 tokenize, part of speech tagging, 이후 명사만 추출하는 과정을 거친 것과 같습니다. 

In [6]:
def custom_tokenize(doc):    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in noun_set:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

In [7]:
corpus.iter_sent = True

for num_sent, sent in enumerate(corpus):    
    if num_sent == 5:
        break
    print('sentence: ', sent)
    print('nouns: ', custom_tokenize(sent), '\n\n')

sentence:  (의왕 국회사진기자단=연합뉴스) 김성태 '최순실 국조특위' 위원장이 26일 오전 경기도 의왕시 서울구치소에서 열린 현장청문회에 입장하고 있다. 2016.12.26
nouns:  ['국회', "'최순실", '위원장', '26일', '오전', '경기', '서울구치소', '현장청문회', '입장'] 


sentence:  scoop@yna.co.kr
nouns:  [] 


sentence:  (서울=연합뉴스) 이재희 기자 = 소녀시대의 태연이 26일 오후 서울 강남구 코엑스에서 열린 'SBS 어워즈 페스티벌(SAF) 가요대전'에서 포즈를 취하고 있다. 2016.12.26
nouns:  ['기자', '26일', '오후', '강남구'] 


sentence:  scape@yna.co.kr
nouns:  [] 


sentence:  영국 팝스타 조지 마이클 별세. [EPA=연합뉴스 자료사진]
nouns:  ['영국', '조지', '마이클', '별세', '자료'] 




sklearn의 CountVectorizer를 이용하여 term frequency matrix를 만듦니다. 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=custom_tokenize)

corpus.iter_sent = False
x = vectorizer.fit_transform(corpus)

x

<1355x1605 sparse matrix of type '<class 'numpy.int64'>'
	with 38327 stored elements in Compressed Sparse Row format>

## Classification을 위한 positive / negative set 만들기

국회라는 단어가 들어간 문서와 그렇지 않은 문서로 1355개의 문서의 class를 나눈 뒤, 이를 구분할 수 있는 classifier를 학습하겠습니다. 

In [9]:
aspect_word = '국회'
aspect_id = vectorizer.vocabulary_.get(aspect_word, -1)

print('%s id = %d' % (aspect_word, aspect_id))
print('num of doc (%s) = %d' % (aspect_word, len(x[:,aspect_id].nonzero()[0])))

국회 id = 298
num of doc (국회) = 210


sparse matrix는 nonzero()라는 함수가 있으며, matrix에서 값이 0이 아닌 위치를 (row id list, column id list)로 나타내줍니다. aspect_id에 해당하는 컬럼을 떼어냈기 때문에 submatrix의 모양이 (1355, 1)이 되었습니다

In [10]:
print(x[:,aspect_id].shape)

from pprint import pprint
pprint(x[:,aspect_id].nonzero())

(1355, 1)
(array([   0,    4,    5,   14,   23,   24,   28,   33,   44,   45,   48,
         50,   51,   58,   59,   63,   67,   72,   81,   87,   96,  103,
        104,  105,  114,  124,  125,  129,  135,  136,  146,  150,  151,
        166,  171,  174,  179,  186,  191,  195,  199,  205,  206,  216,
        226,  248,  255,  262,  266,  273,  282,  285,  292,  298,  311,
        353,  358,  362,  381,  384,  388,  428,  438,  445,  450,  462,
        466,  467,  472,  473,  477,  493,  495,  501,  508,  515,  517,
        536,  545,  552,  556,  557,  560,  561,  564,  566,  574,  575,
        596,  606,  607,  612,  613,  629,  630,  638,  640,  642,  650,
        652,  674,  682,  684,  685,  690,  695,  698,  700,  707,  709,
        711,  713,  723,  725,  728,  729,  737,  743,  744,  748,  752,
        762,  772,  773,  775,  779,  781,  782,  791,  792,  793,  843,
        845,  849,  851,  852,  856,  864,  867,  871,  876,  880,  881,
        890,  893,  898,  903,  911,  91

'국회'란 단어가 들어간 문서들의 리스트를 sparse matrix에서 가져왔습니다. 이를 이용하여 각 문서에 '국회'라는 단어가 들어있으면 1, 아니면 -1인 label list y를 만듦니다

In [11]:
pos_idx = set(x[:,aspect_id].nonzero()[0]) # in 함수는 list보다 set이 빠릅니다
y = [1 if i in pos_idx else -1 for i in range(x.shape[0])]

print('x shape = %s, len(y) = %d, num_pos = %d' % (str(x.shape), len(y), len(pos_idx)))

x shape = (1355, 1605), len(y) = 1355, num_pos = 210


## Logistic regression을 이용한 문서 분류기

#### LogisticRegression의 constructor

- penalty에 l2나 l1을 넣을 수 있으며, 이 때, C는 강의자료의 lambda에 해당합니다. 
- fit_intercept는 데이터의 평행이동을 해주는 중요한 페러매터이므로 True로 그대로 둡니다. 
- 학습이 될 수 있는지 debugging 용으로 확인하고 싶을 때에는 max_iter를 잠시 줄여도 좋습니다. 
- n_jobs는 동시에 여러 개의 cpu processor를 쓸 수 있도록 하는 방법입니다. 절대 수업시에는 n_jobs를 2 이상으로 설정하지 마십시요. 모든 이들이 한 번에 돌리면 CPU가 남아나질 않습니다. 코어가 여러 개인 컴퓨터에서는 n_jobs를 크게 잡으시면 빠르게 계산이 됩니다. 


## L2 regulization을 이용한 문서 판별기 학습

국회라는 단어가 들어간 문서와 들어가지 않는 문서를 분류하기 위한 예제이므로, x_train이라는 새로운 sparse term frequency matrix를 만든 뒤, 국회 단어의 빈도수를 0으로 바꿨습니다. 왜냐하면, '국회'라는 단어 자체가 y label이기 때문입니다. 

In [14]:
from sklearn.linear_model import LogisticRegression

x_train = vectorizer.fit_transform(corpus)
x_train[:,aspect_id] = 0

logistic_l2 = LogisticRegression(penalty='l2')
logistic_l2.fit(x_train, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

위 정확도는 training error를 보기 위함이며, test set에 의한 일반화 성능 및 (Cross validation)은 나중에 다루도록 하겠습니다. training error가 0.002이므로, 1355개의 문서를 '국회'가 들어간 문서와 들어가지 않은 문서로 거의 완벽히 나눌 수 있습니다. 국회라는 단어의 유무를 알려주는 다른 단어들이 존재한다는 의미입니다. 

In [15]:
y_pred = logistic_l2.predict(x)
print(y_pred.shape)

(1355,)


In [16]:
accuracy = sum([1 if pred == answ else 0 for pred, answ in zip(y_pred, y)]) / len(y_pred)
print('training error = %.3f' % (1 - accuracy))

training error = 0.002


## regression coefficients 뜯어보기

vectorizer의 vocabulary_의 크기와 logistic_l2.coef_의 크기가 같습니다. coef는 각 단어의 positive class에 대한 기여도입니다. 

In [18]:
print('logistic regression coefficient:',logistic_l2.coef_.shape)
print('size of vocabulary:', len(vectorizer.vocabulary_))

logistic regression coefficient: (1, 1605)
size of vocabulary: 1605


matrix.reshape(-1)을 하면 matrix를 vector 형태로 바꿔줍니다

In [19]:
logistic_l2.coef_.reshape(-1).shape

(1605,)

vectorizer.vocabulary_ 로부터 각 차원이 어떤 단어에 해당하는지 확인할 수 있는 index2word list를 만듦니다. 

In [21]:
index2word = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
index2word = [word for word, i in index2word]
print(index2word[50:55])

['2003년', '2007년', '2008년', '2010년', '2011년']


In [23]:
for i, v in enumerate(logistic_l2.coef_.reshape(-1)):
    print('%10s : %.3f' % (index2word[i], v))
    if i == 10: break

        "국 : -0.056
       "국민 : 0.037
        "그 : -0.173
        "내 : 0.053
       "내년 : -0.004
        "대 : -0.195
      "대통령 : -0.024
       "미국 : -0.010
        "반 : -0.002
        "새 : 0.016
        "시 : -0.014


coef에 대하여 그 크기가 큰 순서대로 sorting을 하여 살펴보겠습니다. 

회의, 원내대표, 최고, 당대 등의 단어가 들어간 문서는 '정부'라는 단어가 들어있는 문서일 가능성이 높습니다. 

In [24]:
coef_l2 = logistic_l2.coef_.reshape(-1)
beta_l2 = [(index2word[i], coef) for i, coef in enumerate(coef_l2)]
beta_l2 = sorted(beta_l2, key=lambda x:x[1], reverse=False)
beta_l2[:20]

[('모습', -1.0404240735708088),
 ('브리핑', -1.0388715549548639),
 ('의원들', -1.0089950006535855),
 ('비롯', -0.97448753355065421),
 ('중진회의', -0.85295739852841712),
 ('호남', -0.77340533840362102),
 ('시장', -0.69579662596732228),
 ('강남구', -0.69541565035601161),
 ('서구', -0.66699495102239281),
 ('비상대책위원장', -0.66191213893298972),
 ('대화', -0.65380745408649599),
 ('수감동', -0.61881136799606529),
 ('기자', -0.6040897668702816),
 ('오후', -0.58671000316410393),
 ('종로구', -0.56681772516425721),
 ('예방', -0.50928896296021542),
 ('선보', -0.50019149270148655),
 ('비주류', -0.49898992258268715),
 ('부산', -0.49668661257813423),
 ('관계장관회의', -0.49117958851750571)]

## L1 (LASSO)를 이용한 keyword extraction

앞의 예제에서는 L2 regularization을 하였기 때문에 많은 단어들이 모두 고려되었습니다. 이번에는 L1 regularization을 이용합니다. 이는 국회가 들어간 문서와 아닌 문서를 구분하기 위한 최소한의 features (= nouns)를 선택하는 효과가 있습니다. 

즉, 여기서 정의하는 키워드의 기준은, 하나의 class set과 다른 class set을 구분할 수 있는 최소한의 단어집합입니다. 키워드를 추출하기 위함이니 이번에는 x_train 대신, 국회라는 단어가 포함되어 있는 term frequency matrix인 x를 이용하여 학습합니다. 

In [25]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1')
logistic_l1.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

이번에는 L1 의 regularization cost에 따른 키워드 추출의 경향을 살펴보겠습니다. cost 별로 beta의 크기가 큰 top 50개의 키워드들을 top50 dict에 저장해둡니다. 

costs는 1/C 의 값입니다. C = 500이면 lambda = 1/500이므로, regularization이 매우 작게 됨을 의미합니다. 

In [26]:
costs = [500, 200, 100, 50]
top50 = {} 

for cost in costs:
    logistic_l1 = LogisticRegression(penalty='l1', C=cost)
    logistic_l1.fit(x, y)
    
    coef_l1 = logistic_l1.coef_.reshape(-1)
    beta_l1 = [(index2word[i], coef) for i, coef in enumerate(coef_l1)]
    beta_l1 = sorted(beta_l1, key=lambda x:x[1], reverse=True)
    top50[cost] = beta_l1[:50]
    print('done with cost = %.1f' % cost)

done with cost = 500.0
done with cost = 200.0
done with cost = 100.0
done with cost = 50.0


Regularization이 강하게 되면서, 국회라는 단어가 들어간 문서집합을 설명하기 위해 선택한 단어의 개수가 줄어듦니다. 이를 국회라는 문서가 들어간 문서집합의 키워드로 추출할 수 있습니다. 

In [36]:
''.join(['a', 'b', 'c'])

'abc'

In [31]:
for top in range(50):
    message = '\t'.join(['%10s' % (top50[cost][top][0] if top50[cost][top][1] > 0.001 else '') 
                         for cost in costs])
    print(message)
        

        국회	        국회	        국회	        국회
        당대	      원내대표	      원내대표	        의원
      '최순실	        의원	        의원	       청문회
        회의	        대표	       청문회	        대표
      원내대표	        최고	        대표	        오전
        고발	       청문회	        의혹	        실장
        최고	        사표	     서울구치소	        의혹
     창당추진위	        최씨	        실장	       최순실
    더불어민주당	      국정조사	        오전	        자택
       구치소	       최순실	       최순실	      원내대표
        회동	     서울구치소	        질문	          
        사표	        실장	        자택	          
        의결	        자택	      압수수색	          
        현안	     창당추진위	       청와대	          
        축사	        오전	          	          
        이혼	       문체부	          	          
     서울구치소	        회의	          	          
       민간인	        업무	          	          
        의원	      문화예술	          	          
      국정조사	        재산	          	          
     기자간담회	      국민의당	          	          
        입장	          	          	          
      새누리당	          	          